In [2]:
import os,sys
from glob import glob
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image


img_dir = "/root/autoencoder_denoiser/dataset/real_img_referral_for_testing"
clean_dir = os.path.join(img_dir, "real_hsqc_clean")
noisy_dir = os.path.join(img_dir, "real_hsqc_noisy")
imgs=[]
all_img_paths = sorted(glob(noisy_dir+"/*"))

last_compound_name = ''
for img_path in tqdm(all_img_paths):
    compound_name = img_path.split('/')[-1].split('_')[0]
    if compound_name!=last_compound_name:
        save_dir = f'/root/autoencoder_denoiser/dataset/group_by_name_and_stage/{compound_name}/'
        os.makedirs(save_dir, exist_ok=True)
        last_compound_name = compound_name
        '''ground truth'''
        ground_path = img_path.replace("real_hsqc_noisy","real_hsqc_clean").replace("_noisy","_original")
        ground_path = ground_path[:-5] + '1.png'
        if ground_path[-6].isdigit():
            # print(ground_path)
            ground_path = ground_path[:-6]+ground_path[-5:]
        # print(gound_path)
        img = Image.open(ground_path)
        img = np.array(img)
        plus_groud = 1-img[:,:, 0]/255 # I will assume this is plus but not sure
        minus_groud = 1-img[:,:, 2]/255
        ground_truth = plus_groud - minus_groud
        ground_truth = cv2.resize(ground_truth.astype("float32"), (120, 180))
        np.save(save_dir+'ground_truth.npy', ground_truth)
    
    '''noise'''
    img = Image.open(img_path)
    img = np.array(img)
    plus = 1-img[:,:, 0]/255 # I will assume this is plus but not sure
    minus = 1-img[:,:, 2]/255
    # print(plus.shape)
    img_result = plus-minus
    resized_input = cv2.resize(img_result.astype("float32"), (120, 180))
    noise_level = img_path.split('/')[-1].split('_')[-1]
    np.save(save_dir+f'_noise_{noise_level}.npy', resized_input)

100%|██████████| 60/60 [03:31<00:00,  3.53s/it]


In [8]:
import pickle
all_names=[]
for img_path in tqdm(all_img_paths):
    compound_name = img_path.split('/')[-1].split('_')[0]
    if compound_name not in all_names :
        all_names.append(compound_name)
        
with open('/root/autoencoder_denoiser/dataset/all_names.pkl', 'wb') as f:
    pickle.dump(all_names, f)

100%|██████████| 60/60 [00:00<00:00, 284681.27it/s]


In [4]:
with open('/root/autoencoder_denoiser/dataset/imgs_as_array.pkl', 'rb') as f:
        img_loaded =  pickle.load(f)
        

In [9]:
all_names

['Artemisinin',
 'Cyclosporin A',
 'Digoxin',
 'Erythromycin',
 'Menthol',
 'Palmitic acid',
 'Quercetin',
 'Rapamycin',
 'Reserpine',
 'Rifampicin',
 'Strychnine']

In [7]:
a = ["fr", '_1']
sorted(a)

['_1', 'fr']